# Load OSM data to add more info to the flow record

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import subprocess
import sys
def get_repo_root():
    """Get the root directory of the repo."""
    dir_in_repo = os.path.dirname(os.path.abspath('__file__')) # os.getcwd()
    return subprocess.check_output('git rev-parse --show-toplevel'.split(),
                                   cwd=dir_in_repo,
                                   universal_newlines=True).rstrip()
sys.path.append(get_repo_root())
ROOT_dir = get_repo_root()

In [2]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import ast
import numpy as np
osm_folder = ROOT_dir + '/dbs/osm/'
original_network = ROOT_dir + '/dbs/osm/Road_Network_Istanbul.shp'
city = 'istanbul'

## 1 Load the original road network

In [4]:
gdf = gpd.read_file(original_network)
minx, miny, maxx, maxy = gdf.geometry.total_bounds
gdf['osm_id'] = gdf['osm_id'].astype(int)
gdf['osm_id'] = gdf['osm_id'].astype(str)
gdf.columns

Index(['osm_id', 'name', 'ref', 'type', 'oneway', 'bridge', 'maxspeed',
       'geometry'],
      dtype='object')

## 2 Download the complete OSM and save it

In [5]:
new_network = osm_folder + city + '.graphml'
if not os.path.exists(new_network):
    G = ox.graph_from_bbox(maxy, miny, maxx, minx, network_type='drive')
    ox.save_graphml(G, filepath = osm_folder + city + '.graphml')
else:
    G = ox.load_graphml(new_network)

## 3 Process the downloaded network

In [6]:
gdf_n = ox.graph_to_gdfs(G)
edge = gdf_n[1]
edge = edge.loc[:, ['geometry', 'highway', 'junction', 'length', 'maxspeed', 'name', 'oneway',
                    'osmid', 'u', 'v', 'width', 'lanes']]
fields = ['osmid', 'width', 'lanes']
df_inter = pd.DataFrame()
for f in fields:
    df_inter[f] = edge[f].astype(str)
gdf_edge = gpd.GeoDataFrame(df_inter,geometry=edge["geometry"])
gdf_edge = gdf_edge.rename(columns={'osmid': 'osm_id'})

In [7]:
# Create a dict
gdf_edge['osm_id'] = gdf_edge['osm_id'].apply(lambda x: str(ast.literal_eval(x)[0]) if '[' in x else x)
gdf_edge['lanes'] = gdf_edge['lanes'].apply(lambda x: str(ast.literal_eval(x)[0]) if '[' in x else x)
#gdf_edge = gdf_edge.loc[gdf_edge['osm_id'].isin(gdf['osm_id']), :]
gdf_edge.drop_duplicates(subset=['osm_id'], inplace=True)
id2info = {}
for _, row in gdf_edge.iterrows():
    id2info[row['osm_id']] = (row['width'], row['lanes'])

## 4 Update the network with the infor of width, lanes, and oneway

In [8]:
gdf.loc[:, 'width'] = gdf.loc[:, 'osm_id'].apply(lambda x: id2info[x][0] if x in id2info else np.nan)
gdf.loc[:, 'lanes'] = gdf.loc[:, 'osm_id'].apply(lambda x: id2info[x][1] if x in id2info else np.nan)
gdf.head()

,osm_id,name,ref,type,oneway,bridge,maxspeed,geometry,width,lanes
0,4341858,None,None,motorway_link,1,0,0,"LINESTRING (28.69000 41.04111, 28.69020 41.041...",nan,nan
1,4369344,None,D100,trunk,1,0,0,"LINESTRING (29.09750 40.97816, 29.09649 40.979...",nan,nan
2,4406227,Celal Atik Sokak,None,residential,0,0,0,"LINESTRING (28.99528 41.05788, 28.99476 41.057...",nan,nan
3,4412402,Edirne-İstanbul Otoyolu,O-3,motorway,1,0,0,"LINESTRING (28.79442 41.06402, 28.79347 41.064...",nan,nan
4,4434314,None,None,tertiary,0,0,0,"LINESTRING (28.99528 41.06387, 28.99532 41.063...",nan,nan


In [10]:
gdf.to_file(osm_folder + city + ".shp")
#gdf_edge.to_file(osm_folder + city + ".shp")